In [34]:
import openai
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

In [35]:
load_dotenv()
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [36]:
class Infantil:
    
    def __init__(self, date, date_str):
        self.date = date
        self.date_str = date_str
        self.theme = None
        self.output_area = widgets.Output()
        self.create_dropdown()
        
        self.news1_image = ''
        self.news1_header = ''
        self.news1_text = ''
        
        self.news2_image = ''
        self.news2_header = ''
        self.news2_text = ''
        
        self.news3_image = ''
        self.news3_header = ''
        self.news3_text = ''
    
    
    def get_theme_options(self):
        url = 'https://www.ponteiro.com.br/todas_datas.php'
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_dates = soup.find_all('tr')
        
        calendar = []
        edition_str = self.date.strftime('%d/%m')
        for date in all_dates:
            day = date.find('td', width=" 40").get_text().strip() if date.find('td', width=" 40") != None else None
            val = date.find('td', width="490").get_text().strip() if date.find('td', width="490") != None else None
            if day == edition_str and val != None:
                calendar.append({"dia": day, "comemora": val})
        return calendar


    def create_dropdown(self):
        theme_options = self.get_theme_options()
        options = ['Personalizar tema']
        for t in theme_options:
            options.append(t["comemora"])
        self.theme_dropdown = widgets.Dropdown(
            options=options,
            description='Tema do dia:',
            disabled=False,
        )
        self.confirm_button = widgets.Button(
            description='Confirmar',
            button_style='success',
            tooltip='Clique para confirmar o tema selecionado',
            icon='check'
        )
        self.confirm_button.on_click(self.on_confirm_button_click)
        display(self.theme_dropdown, self.confirm_button, self.output_area)
    
    
    def on_confirm_button_click(self, b):
        with self.output_area:
            # clear_output()
            self.theme = self.theme_dropdown.value
            if self.theme == 'Personalizar tema':
                self.text_input = widgets.Text(
                    description='Tema personalizado:',
                    placeholder='Digite o tema aqui...',
                )
                self.submit_button = widgets.Button(
                    description='Submeter',
                    button_style='info',
                    tooltip='Clique para submeter o tema personalizado',
                    icon='check'
                )
                self.submit_button.on_click(self.on_submit_button_click)
                display(self.text_input, self.submit_button)
            else:
                self.setup_news1()
    
    
    def on_submit_button_click(self, b):
        with self.output_area:
            # clear_output()
            self.theme = self.text_input.value
            self.setup_news1()


    def setup_news1(self):
        print(f"\nO tema escolhido foi: {self.theme}")
        print(f'Infantil - Notícia 1')
        parts = self.date_str.split(' ')
        # self.get_news1_text()
        # Imprimir imagem
        self.news1_header = f'{parts[1]} de {parts[3]}: {self.theme}'
        print(f'Header: {self.news1_header}\n')
        self.get_news1_text()
        print(f'Text: {self.news1_text}\n')
        self.ask_news2_header()
    
    
    def get_news1_text(self):
        content = f'Escreva um texto voltado para crianças recém alfabetizadas com o tema {self.theme}. ' 
        content += f'Esse texto estará na seção infantil do dia {self.date.strftime("%d/%m")} ao lado de uma imagem em preto em branco (para ser colorida) '
        content += f'sobre o tema. O texto deve conter entre 900 e 950 caracteres. Escreva estritamente o corpo do texto.'
        
        client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
        message = [{
            'role': 'user',
            'content': content
        }]
        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=10000, 
            temperature=0, 
        )
        news_text = response.choices[0].message.content
        self.news1_text = news_text
    
    
    def ask_news2_header(self):
        self.news2_input = widgets.Text(
            description='Tema para News 2:',
            placeholder='Digite o tema da primeira curiosidade aqui...',
        )
        self.news2_submit_button = widgets.Button(
            description='Gerar News 2',
            button_style='info',
            tooltip='Clique para gerar a segunda notícia',
            icon='check'
        )
        self.news2_submit_button.on_click(self.on_news2_submit_button_click)
        display(self.news2_input, self.news2_submit_button)
    
    
    def on_news2_submit_button_click(self, b):
        with self.output_area:
            # clear_output()
            self.news2_header = self.news2_input.value
            self.setup_news2()
    
    
    def setup_news2(self):
        print(f'Infantil - Notícia 2')
        print(f"\nTema da primeira curiosidade: {self.news2_header}")
        
        self.news2_text = self.get_curiosity_text(self.news2_header)
        # self.news2_image = self.get_curiosity_image(self.news2_text)
        # Imprimir a imagem
        print(f'Text: {self.news2_text}\n')
        self.ask_news3_theme()




    
    
    def ask_news3_theme(self):
        self.news3_input = widgets.Text(
            description='Tema para News 3:',
            placeholder='Digite o tema da segunda curiosidade aqui...',
        )
        self.news3_submit_button = widgets.Button(
            description='Gerar News 3',
            button_style='info',
            tooltip='Clique para gerar a segunda notícia',
            icon='check'
        )
        self.news3_submit_button.on_click(self.on_news3_submit_button_click)
        display(self.news3_input, self.news3_submit_button)
    
    
    def on_news3_submit_button_click(self, b):
        with self.output_area:
            # clear_output()
            self.news3_header = self.news3_input.value
            print(f"\nTema para News 3: {self.news3_header}")
            self.setup_news3()
    
    
    def setup_news3(self):
        print(f'Infantil - Notícia 3')
        print(f"\nTema da primeira curiosidade: {self.news3_header}")
        
        self.news3_text = self.get_curiosity_text(self.news3_header)
        # self.news3_image = self.get_curiosity_image(self.news3_text)
        # Imprimir a imagem
        print(f'Text: {self.news3_text}\n')
        
        

    
        
    def get_curiosity_text(self, theme):
        content = f'Aponte-me uma curiosidade para inserir na seção infantil de um jornal. Forneça, como resposta, estritamente a curiosidade da maneira como ela deve ser escrita em um texto contento entre 800 e 850 caracteres. O tema é {theme}.'
        client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))
        message = [{
            'role': 'user',
            'content': content
        }]
        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=10000, 
            temperature=1, 
        )
        news_text = response.choices[0].message.content
        print(f'Infantil - Curiosidade: {theme}')
        return news_text
    
    
    
    def get_curiosity_image(self, curiosity):
        prompt = f'Crie uma imagem infantil para ser colocara junto ao texto a seguir na seção infantil de um jornal de grande circulação.\n\n' 
        prompt += f'{curiosity}'
        
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        image_url = response.data[0].url
        return image_url

